In [1]:
pip install sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import random
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Function to clean text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # remove special characters
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = text.lower().strip()  # lowercase and strip whitespace
    return text

# Load and clean data
df = pd.read_csv("data.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[df['Sentiment'].isin(['positive', 'neutral', 'negative'])]
df['Sentence'] = df['Sentence'].apply(clean_text)

# Label mapping
labels = df['Sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2}).values

# Sentence embeddings using Sentence Transformers
model_name = 'all-distilroberta-v1'
embedder = SentenceTransformer(model_name)
embeddings = embedder.encode(df['Sentence'].tolist(), show_progress_bar=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# DQN Parameters
state_size = len(embeddings[0])
num_actions = 3
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
gamma = 0.95
lr = 0.001
batch_size = 64
memory = deque(maxlen=2000)

# DQN Model
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)

# Initialize main and target models
model = DQN(state_size, num_actions)
target_model = DQN(state_size, num_actions)
target_model.load_state_dict(model.state_dict())
target_model.eval()

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss()

# Epsilon-greedy policy
def act(state):
    global epsilon
    if np.random.rand() <= epsilon:
        return random.randrange(num_actions)
    state = torch.FloatTensor(state).unsqueeze(0)
    with torch.no_grad():
        q_values = model(state)
    return torch.argmax(q_values).item()

# Replay function using Double DQN
def replay():
    global epsilon
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        state = torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state)
        with torch.no_grad():
            next_action = torch.argmax(model(next_state)).item()
            target_q = target_model(next_state)[next_action]
            target = reward if done else reward + gamma * target_q.item()

        current_qs = model(state)
        target_qs = current_qs.clone()
        target_qs[action] = target

        optimizer.zero_grad()
        loss = loss_fn(current_qs, target_qs.detach())
        loss.backward()
        optimizer.step()

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Training loop
episodes = 20
target_update_freq = 5
for e in range(episodes):
    for i in range(len(X_train)):
        state = X_train[i]
        action = act(state)
        reward = 1 if action == y_train[i] else -1
        done = True
        next_state = X_train[i]
        memory.append((state, action, reward, next_state, done))
        replay()

    if e % target_update_freq == 0:
        target_model.load_state_dict(model.state_dict())

    print(f"Episode {e+1}/{episodes} completed. Epsilon: {epsilon:.2f}")

# Evaluation
correct = 0
total = len(X_test)
for i in range(total):
    state = torch.FloatTensor(X_test[i]).unsqueeze(0)
    with torch.no_grad():
        pred = torch.argmax(model(state)).item()
    if pred == y_test[i]:
        correct += 1

print(f"Test Accuracy: {100 * correct / total:.2f}%")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\ayush\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayush\.cache\huggingface\hub\models--sentence-transformers--all-distilroberta-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

Episode 1/20 completed. Epsilon: 0.01
